In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import math
warnings.filterwarnings(action='ignore')

%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

plt.rcParams["font.family"] = 'NanumSquareRoundOTF'
plt.rcParams["font.size"] = 12

In [2]:
df_test = pd.read_csv('../Data/test_PCA_jmk_lnd.csv')
df_train = pd.read_csv('../Data/train_PCA_jmk_lnd.csv')
df_valid = pd.read_csv('../Data/valid_PCA_jmk_lnd.csv')
# 원본
df_te = pd.read_csv('../Data/PJT002_test.csv')
df_tr = pd.read_csv('../Data/PJT002_train.csv')
df_va = pd.read_csv('../Data/PJT002_validation.csv')

In [37]:
# 데이터 리스트
df_l = [df_train, df_test, df_valid]

# 자주 쓸 컬럼들
w = ['id', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 'ttl_grnd_flr', 'ttl_dwn_flr','lnd_us_sttn_nm(clean)']
u = ['id', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 'ttl_grnd_flr', 'ttl_dwn_flr','lnd_us_sttn_nm']

## 01. 지하 층 결측치 채우기

In [4]:
# 지하 층 컬럼 복원하기
# 지하 층 수 notull은 평균 0.13... 이므로 0으로 채움

a = df_te[['id','ttl_dwn_flr']].fillna(0)
df_test = df_test.merge(a, on='id')

a = df_tr[['id','ttl_dwn_flr']].fillna(0)
df_train = df_train.merge(a, on='id')

a = df_va[['id','ttl_dwn_flr']].fillna(0)
df_valid = df_valid.merge(a, on='id')

## 02. 지상 층 결측치 채움

In [5]:
# 지상 층수는 상황에 나누어 채움
# 지워진 층수 데이터 합치고 채우기
def grnd_n(row) :
        # 0이지만 0이 아니어야 할 때
        if (row['ttl_grnd_flr'] == 0) & (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0):
            return math.ceil(row['ttl_ar'] / row['bldng_ar']) 
        # null이 아니라면 있는 그대로
        elif pd.notnull(row['ttl_grnd_flr']) :
            return row['ttl_grnd_flr']
        # null : 연면적 /빌딩면적 - 지하층수 
        elif (row['ttl_ar'] != 0) & (row['bldng_ar'] != 0):
            return math.ceil(row['ttl_ar'] / row['bldng_ar'])
        # ttl_ar나 bldng_ar가 0이라면 0, axis=1)
        return 0


#지상층 컬럼 다시 불러오기
a = df_te[['id', 'ttl_grnd_flr']]
df_test = df_test.merge(a, on='id')
df_test['ttl_grnd_flr'] = df_test.apply(grnd_n, axis=1)


a = df_tr[['id','ttl_grnd_flr']]
df_train = df_train.merge(a, on='id')
df_train['ttl_grnd_flr'] = df_train.apply(grnd_n, axis=1)


a = df_va[['id','ttl_grnd_flr']]
df_valid = df_valid.merge(a, on='id')
df_valid['ttl_grnd_flr'] = df_valid.apply(grnd_n, axis=1)

In [6]:
# 결측치가 얼마나 남았는지 체크하는 함수
def print_null(df):
    print('건물채수가 0인 데이터: ', len(df[df['bldng_cnt'] == 0]), '개')
    print('건축면적이 0인 데이터: ', len(df[df['bldng_ar'] == 0]), '개')
    print('건물연면적 0인 데이터: ', len(df[df['ttl_ar'] == 0]), '개')
    print('토지면적이 0인 데이터: ', len(df[df['lnd_ar'] == 0]), '개')        
    print('지상 층수 데이터 없는 데이터: ', len(df[df['ttl_grnd_flr'].isnull()]), '개')
    print('지하 층수 데이터 없는 데이터: ', len(df[df['ttl_dwn_flr'].isnull()]), '개')
    print('지상 층수가 0 : ', len(df[df['ttl_grnd_flr'] == 0]), '개')
    print('지하 층수가 0: ', len(df[df['ttl_dwn_flr'] == 0.0]), '개')
    
print_null(df_tr)

건물채수가 0인 데이터:  0 개
건축면적이 0인 데이터:  9660 개
건물연면적 0인 데이터:  8426 개
토지면적이 0인 데이터:  25569 개
지상 층수 데이터 없는 데이터:  10210 개
지하 층수 데이터 없는 데이터:  11005 개
지상 층수가 0 :  103 개
지하 층수가 0:  44579 개


In [7]:
# df_tr[df_tr['ttl_grnd_flr'] == 0][u]

,id,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,ttl_grnd_flr,ttl_dwn_flr,lnd_us_sttn_nm
644,94401,7,37.20,37.20,0.0,0.0,0.0,단독
1291,71498,5,43.80,43.80,0.0,0.0,0.0,단독
1739,96376,5,77.50,77.50,0.0,0.0,0.0,단독
5081,136559,1,140.21,271.32,0.0,0.0,0.0,단독
8354,87053,6,118.60,118.60,0.0,0.0,0.0,단독
8420,97121,4,113.50,113.50,943.0,0.0,0.0,단독
9494,75519,1,211.10,270.48,628.0,0.0,0.0,단독
9503,106821,1,145.50,145.50,0.0,0.0,0.0,주거기타
9647,132906,2,2625.00,2625.00,13376.0,0.0,0.0,답
9701,102766,3,88.80,131.70,180.0,0.0,2.0,단독


## 03. 빌딩면적 결측치

### 연면적, 빌딩면적, 토지면적이 모두 있는 데이터를 기준으로 결측치를 채운다.


### 토지 사용도에 따라 많은 데이터의 기준점이 달라질 것으로 예상하고, 어떤 데이터들이 비어있는지 확인해 보자

In [12]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
us_cnt = pd.DataFrame()


d = D[A & B & C]
us_cnt['nnn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & B & ~C] 
us_cnt['nny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & ~C] 
us_cnt['nyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & C]
us_cnt['nyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & C] 
us_cnt['ynn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & C] 
us_cnt['yyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & ~C] 
us_cnt['yny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & ~C]
us_cnt['yyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

us_cnt['sum'] = us_cnt.apply(lambda row : row.sum(), axis = 1)
us_cnt.loc['sum'] = us_cnt.apply(lambda col : col.sum())
us_cnt.tail()

,nnn,nny,nyy,nyn,ynn,yyn,yny,yyy,sum
lnd_us_sttn_nm(clean),,,,,,,,,
주차장등,12.0,NaN,NaN,NaN,NaN,12.0,NaN,26.0,50.0
토지임야,75.0,NaN,NaN,NaN,3.0,19.0,NaN,28.0,125.0
특수기타,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0
하천등,328.0,NaN,NaN,NaN,NaN,20.0,NaN,20.0,368.0
sum,8062.0,342.0,13.0,8.0,929.0,16555.0,326.0,32924.0,59159.0


<br>
<br>

### 토지면적이 모두 있는 데이터 안에서 컬럼간의 상관관계를 보자

In [13]:
df_tr_nn = df_train[~A & ~B & ~C]

In [14]:
df_tr_nn[w].corr().iloc[2:,2:]

,bldng_ar,ttl_ar,lnd_ar,ttl_grnd_flr,ttl_dwn_flr
bldng_ar,1.000000,0.912741,0.769379,0.553462,0.686816
ttl_ar,0.912741,1.000000,0.881332,0.666529,0.802816
lnd_ar,0.769379,0.881332,1.000000,0.439224,0.663700
ttl_grnd_flr,0.553462,0.666529,0.439224,1.000000,0.735284
ttl_dwn_flr,0.686816,0.802816,0.663700,0.735284,1.000000


<br>

### 토지 사용도에 따른 컬럼의 표준편차를 살펴본다.

In [19]:
t = pd.DataFrame()
use_l = df_tr_nn['lnd_us_sttn_nm(clean)'].unique()

for use in use_l :
    dt = [round(np.std(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['lnd_ar']), 1),
          round(np.std(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['ttl_ar']), 1),
          round(np.std(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['bldng_ar']),1)]
    t[use] = dt    
ta = np.transpose(t)
ta = ta.reset_index()
ta = ta.rename(columns = {'index': 'lnd_us_sttn_nm(clean)'})
ta = ta.rename(columns={0:'lnd_ar표편', 1:'ttl_ar', 2:'blndg_ar'})
ta

,lnd_us_sttn_nm(clean),lnd_ar표편,ttl_ar,blndg_ar
0,단독,5647.7,1768.0,1261.7
1,상업용,10876.8,10538.1,1681.3
2,주상용,973.8,1218.4,296.4
3,아파트,256665.0,61119.6,8892.5
4,주거기타,9402237.5,84789.0,25935.8
5,주거나지,203448.5,15693.4,1583.1
6,업무용,11108.4,5479.7,1137.9
7,주상기타,3561.8,1258.1,471.6
8,운동장등,122591.4,8666.9,6602.2
9,전기타,5261.6,853.4,683.5


---
<br>

## 빌딩면적 결측값 채우기 (bldng_ar)
#### 1. 연면적이 있고 층수가 1일 이상일 때, 
####       bldng_ar = around(bldng_ar = ttl_ar / 총 층수)

In [20]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
D[C].count()[0]

25567

In [21]:
df = df_train
df['bldng_ar'] = df.apply(lambda row :
                          row['bldng_ar'] if row['bldng_ar'] != 0
                          else row['ttl_ar'] / row['ttl_grnd_flr'] if (row['ttl_grnd_flr'] >= 1) & (row['ttl_ar']!= 0)
                          else row['ttl_ar'] if (row['ttl_ar'] != 0)
                          else row['bldng_ar'] , axis = 1)

df = df_test

df['bldng_ar'] = df.apply(lambda row :
                          row['bldng_ar'] if row['bldng_ar'] != 0
                          else row['ttl_ar'] / (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) if (row['ttl_grnd_flr'] + row['ttl_dwn_flr'] >= 1) & (row['ttl_ar']!= 0)
                          else row['ttl_ar'] if (row['ttl_ar'] != 0)
                          else row['bldng_ar'] , axis = 1)

df = df_valid

def f(row):
    if row['bldng_ar'] != 0:
        return row['bldng_ar']
    if (row['ttl_grnd_flr'] + row['ttl_dwn_flr'] >= 1) & (row['ttl_ar']!= 0):
        return row['ttl_ar'] / (row['ttl_grnd_flr'] + row['ttl_dwn_flr'])
    if (row['ttl_ar'] != 0):
        return row['ttl_ar']
    return row['bldng_ar']

df['bldng_ar'] = df.apply(f, axis = 1)

#### bldng_ar = around(lnd_ar * 상관계수)

DF blm_train

blm = 빌딩면적 / 토지면적 의 평균

tlm = 연면적 / 토지면적의 평균


## 04. 토지면적 결측치

#### 토지면적 아웃라이어(아파트, 목장 등에서 많이 발견)이라고 생각했으나, 아파트와 목장의 용적률은 다른 토지들과 많이 다를 수 있다는 사실이 있음

In [22]:
# # 아웃라이어 삭제
# for df in df_l:
#     df['lnd_ar'] = df.apply(lambda row :
#                         0 if row['lnd_ar'] == 0
#                         else row['lnd_ar'] if (row['lnd_ar'] >= row['ttl_ar']) | (row['lnd_ar'] >= row['bldng_ar'])
#                         else (row['bldng_ar'] * 3) if (row['bldng_ar'] != 0) & (row['lnd_ar'] <= row['bldng_ar'])
#                         else row['lnd_ar']
#                         ,axis = 1)    


In [23]:
# lnd_ar과 상관관계가 높은 변수를 고르는 함수 (building or total)
def find_bort(a, use):
    if len(a) == 0:
        return 't'

    co = a[w].corr()
    t = co.loc['lnd_ar', 'ttl_ar']
    b = co.loc['lnd_ar', 'bldng_ar']
    bort = 't' if t >= b else 'b'
    return bort


# 토지 이용도 마다 최적의 면적대비율을 찾아 표로 만들기
use_l = df_train['lnd_us_sttn_nm(clean)'].unique()

    #train으로만 학습할 것임
blm_train = pd.DataFrame()
df = df_train

for use in use_l :
    a = df[~A & ~B & ~C & (df['lnd_us_sttn_nm(clean)'] == use)]
    
    bl = a['bldng_ar'] / a['lnd_ar']
    tl = a['ttl_ar'] / a['lnd_ar']
    
    blm = round(bl.mean(), 2)
    tlm = round(tl.mean(), 2)

    bort = find_bort(a, use)
#     print('use: ', use)
#     print('bl: ', bl)
#     print('tl: ', tl)
#     print('blm: ', blm)
#     print('tlm: ', tlm)
    s = {'bort': bort, 'tlm': tlm, "blm" : blm, 'lnd_us_sttn_nm(clean)' : use}
    blm_train = blm_train.append(s, ignore_index =True)
    
blm_train = blm_train.fillna(0)

blm_medi = blm_train['blm'].median()
tlm_medi = blm_train['tlm'].median()
print(blm_medi, tlm_medi)

0.27 0.29000000000000004


#### 적용

In [24]:
joined_train = df_train.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
joined_train[['bldng_ar', 'lnd_ar', 'lnd_us_sttn_nm(clean)', 'blm']]
df_train['bldng_ar'] = joined_train.apply(lambda row : row['bldng_ar'] if row['bldng_ar'] != 0
                                               else round(row['lnd_ar'] * row['blm']) if (row['lnd_ar'] != 0) & (row['blm'] != 0)
                                               else 0, axis=1)
# a['bldng_ar'] = a.apply(lambda row :
#                         round(row['lnd_ar'] / blm_train['blm']) if row['lnd_us_sttn_nm(clean)'] == 

In [25]:
print_null(df_train)

건물채수가 0인 데이터:  0 개
건축면적이 0인 데이터:  8062 개
건물연면적 0인 데이터:  8425 개
토지면적이 0인 데이터:  25567 개
지상 층수 데이터 없는 데이터:  0 개
지하 층수 데이터 없는 데이터:  0 개
지상 층수가 0 :  7690 개
지하 층수가 0:  55576 개


#### 특수 기타, 발전소는 데이터 사례가 없기 때문에, 전체 평균으로 결측치를 채운다.

In [26]:
# 데이터 프레임의 lnd_ar가 채워졌는지 확인하는 메서드

def check_yyn(df, use):
    return df[(df['lnd_us_sttn_nm(clean)'] == use) & (df['ttl_ar'] != 0) & (df['bldng_ar'] != 0) & (df['lnd_ar'] == 0)][w]

In [27]:
def see_bl(df, use):
    bort = find_bort(df)
    
    if bort == 'b' : 
        a = df[(df['lnd_us_sttn_nm(clean)'] == use)]
        bl = a['bldng_ar'] / a['lnd_ar']
        blm = bl.mean()
        print(bl.mean())
        print(bl.median())
        sns.distplot(bl)
        return blm
    if bort == 't' :
        a = df[(df['lnd_us_sttn_nm(clean)'] == use)]
        bl = a['ttl_ar'] / a['lnd_ar']
        blm = bl.mean()
        print(bl.mean())
        print(bl.median())
        sns.distplot(bl)
        return blm

# 실행하기
def find_bl(df):
    a, bort = find_bort(df)
    
    if bort == 'b' : 
        bl = a['bldng_ar'] / a['lnd_ar']
        blm = bl.mean()
        return blm
    if bort == 't' :
        bl = a['ttl_ar'] / a['lnd_ar']
        blm = bl.mean()
        return blm

def fill_lnd(df, row, use, blm):
    
    if row['lnd_ar'] != 0:
        return row['lnd_ar']
    
    elif blm == 0 :
        return df['lnd_ar'].median()
    
    # ttl_ar == 0
    elif (row['ttl_ar'] == 0) :
        if row['bldng_ar'] == 0 :
            return 0
        elif row['bldng_ar'] != 0 :
            blm = find_bl(use, 'b')
            return row['bldng_ar'] / blm

    # ttl_ar != 0
    elif row['bldng_ar'] == 0 :
        return row['ttl_ar'] / blm

    # ttl_ar != 0 b_ar != 0
    else :
        return row['bldng_ar'] / blm                                   

In [28]:
joined = df_train.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
len(joined)

59191

In [29]:
#데이터프레임마다 토지사용목적에 따라
df_l = [df_train, df_test, df_valid]

for df in df_l :
    joined = df.merge(blm_train, how = 'left', on='lnd_us_sttn_nm(clean)')
    use_l = df['lnd_us_sttn_nm(clean)'].unique()
    
    df['lnd_ar'] = joined.apply(lambda row: 
                                row['lnd_ar'] if row['lnd_ar'] != 0
                                else 0 if (row['ttl_ar'] == 0) & (row['bldng_ar'] == 0)
                                else row['bldng_ar'] / row['blm'] if (row['bldng_ar'] != 0) & (row['bort'] == 'b') & (row['blm'] != 0) 
                                else row['ttl_ar'] / row['tlm'] if (row['ttl_ar'] != 0) &(row['bort'] == 't') & (row['tlm'] != 0)
                                else row['ttl_ar'] / tlm_medi if (row['ttl_ar'] != 0)
                                else row['bldng_ar'] / blm_medi if (row['bldng_ar'] != 0)
                                else 0 
                                , axis = 1)


## 04. 연면적 결측치 (ttl_ar)

In [30]:
for df in df_l:
    df['ttl_ar'] = df.apply(lambda row :
                            row['ttl_ar'] if row['ttl_ar'] != 0
                            else (row['bldng_ar'] * (row['ttl_grnd_flr'] + row['ttl_dwn_flr']) )if (row['bldng_ar'] != 0)* ((row['ttl_grnd_flr'] + row['ttl_dwn_flr']) >= 1)
                            else row['bldng_ar'] if (row['bldng_ar'] != 0)
                            else 0
                           ,axis = 1)

In [47]:
D = df_train
A = D['ttl_ar'] == 0
B = D['bldng_ar'] == 0
C = D['lnd_ar'] == 0
us_cnt = pd.DataFrame()


d = D[A & B & C]
us_cnt['nnn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & B & ~C] 
us_cnt['nny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & ~C] 
us_cnt['nyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[A & ~B & C]
us_cnt['nyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & C] 
us_cnt['ynn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & C] 
us_cnt['yyn'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & B & ~C] 
us_cnt['yny'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

d = D[~A & ~B & ~C]
us_cnt['yyy'] = d.groupby('lnd_us_sttn_nm(clean)').count()['id']

us_cnt['sum'] = us_cnt.apply(lambda row : row.sum(), axis = 1)
us_cnt.loc['sum'] = us_cnt.apply(lambda col : col.sum())

us_cnt.reset_index()

,lnd_us_sttn_nm(clean),nnn,nny,nyy,nyn,ynn,yyn,yny,yyy,sum
0,고속도로휴게소,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,15.0
1,골프장 대중제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2,골프장 회원제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,8.0
3,공업기타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.0,147.0
4,공업나지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.0,115.0
5,공업용,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2119.0,2119.0
6,공원등,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,46.0
7,공원묘지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
8,과수원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,285.0,285.0
9,기타,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0


-----------

<br>
<br>
<br>
<br>
<br>
<br>


In [ ]:
g = df_train.groupby('month', 'lnd_us_sttn_nm(clean)')[['hmdt', 'tmprtr']]

In [ ]:
from plotly import express as ex
from plotly import graph_objects as go

In [ ]:
g.corr()

In [ ]:
df_train[A & B & C][['bldng_cnt','lnd_us_sttn_nm(clean)', 'fire_yes']]
a = df_train[(df_train['lnd_us_sttn_nm(clean)'] == '답') & df_train['fire_yes'] == 1]
b = df_train[(df_train['lnd_us_sttn_nm(clean)'] == '답') & df_train['fire_yes'] == 0]

In [ ]:
a.groupby('month').count()['id']

In [ ]:
b.groupby('month').count()['id']

In [ ]:
plt.hist(a['month'])

In [ ]:
plt.hist(b['month'])

In [ ]:
print_null(df_tr)
print_null(df_train)

# 면적 데이터가 아에 없는 경우

In [ ]:
count = df_train[(df_train['ttl_ar'] == 0) & (df_train['lnd_ar'] == 0) & (df_train['bldng_ar']==0)].groupby('lnd_us_sttn_nm(clean)')['id'].count()

In [ ]:
count

In [ ]:
b = pd.DataFrame()

for use in use_l :
    dt = [np.std(df_train[df_train['lnd_us_sttn_nm(clean)'] == use]['bldng_ar']),
          np.mean(df_train[df_train['lnd_us_sttn_nm(clean)'] == use]['bldng_ar']),
          np.median(df_train[df_train['lnd_us_sttn_nm(clean)'] == use]['bldng_ar'])]
    
    b[use] = dt

a = np.transpose(b)
a = a.reset_index()
a = a.rename(columns = {'index': 'lnd_us_sttn_nm(clean)'})

### 일단 중앙값으로 모두 채우기

In [41]:
df_l = [df_train, df_test, df_valid]

In [45]:
t = pd.DataFrame()
use_l = df_tr_nn['lnd_us_sttn_nm(clean)'].unique()

for use in use_l :
    dt = [round(np.median(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['lnd_ar']), 1),
          round(np.median(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['ttl_ar']), 1),
          round(np.median(df_tr_nn[df_tr_nn['lnd_us_sttn_nm(clean)'] == use]['bldng_ar']),1)]
    t[use] = dt
ta = np.transpose(t)
ta = ta.reset_index()
ta = ta.rename(columns = {'index': 'lnd_us_sttn_nm(clean)'})
ta = ta.rename(columns={0:'lnd_ar', 1:'ttl_ar', 2:'bldng_ar'})
ta

,lnd_us_sttn_nm(clean),lnd_ar,ttl_ar,bldng_ar
0,단독,330.8,96.5,91.0
1,상업용,441.3,428.2,199.3
2,주상용,244.6,263.1,126.8
3,아파트,5623.5,12278.4,1400.0
4,주거기타,629.0,198.9,162.6
5,주거나지,356.0,136.7,106.5
6,업무용,1101.1,872.5,444.1
7,주상기타,300.0,285.4,139.2
8,운동장등,7962.0,414.0,403.7
9,전기타,969.0,198.0,198.0


In [46]:
def fb(row):
    if row['bldng_ar'] == 0:
        return ta[ta['lnd_us_sttn_nm(clean)'] == row['lnd_us_sttn_nm(clean)']]['bldng_ar']

def ft(row):
    if row['ttl_ar'] == 0:
        return ta[ta['lnd_us_sttn_nm(clean)'] == row['lnd_us_sttn_nm(clean)']]['ttl_ar']

def fl(row):
    if row['lnd_ar'] == 0:
        return ta[ta['lnd_us_sttn_nm(clean)'] == row['lnd_us_sttn_nm(clean)']]['lnd_ar']

    
    
for df in df_l:
    df['bldng_ar'] = df.apply(fb, axis = 1)
    df['ttl_ar'] = df.apply(ft, axis = 1)
    df['lnd_ar'] = df.apply(fl, axis = 1)

In [48]:
# 저장하기

df_test.to_csv('test_PCA_fill_ar.csv', index=False)
df_train.to_csv('train_PCA_fill_ar.csv', index=False)
df_valid.to_csv('valid_PCA_fill_ar.csv', index=False)